# Image Registration and Combination using the JWST Level 3 Pipeline

## Table of Contents:
> * [Imports](#imports)
> * [Introduction](#introduction)
> * [Optional CRDS set up](#crds)
> * [Loading data](#get_data)
>     * [Download Association file](#download_asn)
>     * [Download FITS files](#download_fits)
>     * [Create Configuration Files](#pipeline_configs)
> * [File Information](#file_info)
>     * [Create Association table](#create_association) (optional)
>     * [FITS files](#fits_info)
> * [Methods of calling the pipeline](#methods)
>     * [Run Pipeline with Configuration Files](#pipeline_with_cfgs)
>     * [Run Pipeline with Paramters Set Programmatically](#pipeline_no_configs)
>     * [Run Individual Steps with Configuration Files](#steps_with_config_files)
>     * [Run Individual Steps with Parameters Set Programmatically](#steps_no_configs)
> * [Additional Resources](#addtl_resources)
> * [About this notebook](#about)

***
<a id='imports'></a>
## Imports

* `os` to help import the data used for this notebook
* `requests` to retrieve the data used for this notebook
* `astropy.io fits` for accessing FITS files
* `astropy.table Table` for reading in source catalog files
* `astropy.visualization` simple_norm for creating visualizations of astronomical data
* `matplotlib.pyplot` for plotting data
* `matplotlib inline` to display plots inline

In [ ]:
import os
import requests
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import simple_norm, LogStretch, ImageNormalize, ManualInterval
import matplotlib.pyplot as plt
%matplotlib inline

* jwst.pipeline Image3Pipeline to run the pipeline
* jwst datamodels objects used by the pipelne, which contain astronomical data
* jwst.associations load_asn reads in association file containing a list of FITS files

In [ ]:
# Import pipeline
from jwst.pipeline import Image3Pipeline
from jwst import datamodels
from jwst.associations import load_asn

In [ ]:
# Import individual pipeline steps
from jwst.tweakreg import tweakreg_step
from jwst.skymatch import skymatch_step
from jwst.outlier_detection import outlier_detection_step
from jwst.resample import resample_step
from jwst.source_catalog import source_catalog_step

***
<a id='introduction'></a>
## Introduction

The purpose of this notebook is to illustrate several ways to perform level 3 processing on a set of input imaging data.

Level 3 imaging processing combines the calibrated data from multiple exposures (e.g., dithered or mosaicked data) into a single distortion-corrected product. This is accomplished using the [**calwebb_image3** pipeine](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/description.html#stage-3-imaging-pipeline-step-flow-calwebb-image3) or its constituent steps. Before being combined, the exposures receive additional corrections/calibrations for the purpose of astrometric alignment, background matching, and outlier rejection. See the [Additional Resources](#addtl_resources) section for links to documentation on the calwebb_image3 steps.

> **Inputs**: The inputs to **calwebb\_image3** or its steps will usually be in the form of an [association (ASN) file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/associations/overview.html) that lists multiple associated 2D calibrated exposures (FITS files) to be processed and combined into a single product. The individual exposures should be calibrated (filenames ending in "\_cal.fits") outputs from the [**calwebb\_image2** pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/description.html#stage-2-imaging-pipeline-step-flow-calwebb-image2). It is also possible use a single "*\_cal.fits" file as input, in which case only the **resample** and **source\_catalog** steps of the **calwebb\_image3** pipeline will be applied.

> **Outputs**: A resampled 2D image product with suffix "i2d.fits" is created, containing the rectified (distortion-free) single exposure or the rectified and combined data from the association of exposures. A source catalog produced from the "i2d" product is saved as an ASCII file in "ecsv" format, with a filename suffix of "cat". If the **outlier_detection** step is run, a new version of each input file is created which contains a data quality (DQ) array that has been updated to flag pixels detected as outliers. This updated product is known as a ["cosmic ray-flagged" product](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/description.html?highlight=crf#id14) and is saved to a file with a name matching the input filename but with filetype "crf".

The calibrations performed by the Level 3 pipeline can be run in several different ways. First, there are two ways to call the appropriate pipeline steps:

- Call the pipeline itself
- Call the individual steps that make up the pipeline

With the former method, only a single call is needed to run the entire pipeline with the appropriate steps in the proper order. The latter method allows you to inspect and potentially modify the output from a given step before moving on to the next. 

In addition, there are two methods for calling the pipeline or pipeline steps.

- Using [**configuration files**](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html#configuration-files)
- Setting parameters programmatically

In this notebook, we show examples of all four combinations:

[Methods of calling the level 3 pipeline](#methods)
1. [Call the pipeline with configuration files](#pipeline_with_cfgs)
2. [Call the pipeline with parameters set programmatically](#pipeline_no_configs)
3. [Call the individual pipeline steps using configuration files](#steps_with_config_files)
4. [Call the individual pipeline steps with parameters set programmatically](#steps_no_configs)

***
<a id='crds'></a>
## Optional CRDS Setup

In order to run this notebook, you must have access to the appropriate Calibration Reference Data System (CRDS) files needed by the pipeline. If you are not on the STScI internal network, the easiest way to ensure that you can get the necessary files is to define the two environment variables desribed in the CRDS [offsite use](https://jwst-crds.stsci.edu/static/users_guide/basic_use.html#setup-for-offsite-use) documentation. The **CRDS_SERVER_URL** tells the pipeline where to look for reference files, and the **CRDS_PATH** variable lists where any downloaded reference files should be placed. See the cell below for an example. If you do not have a local copy of the CRDS files, they will be downloaded on an as-needed basis when pipeline steps are executed. If you are on the internal STScI network, this step can be skipped.

- `$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu/ `
- `$ export CRDS_PATH=${HOME}/crds_cache/ `

***
<a id='get_data'></a>
## Loading data

An example dataset to be used with this notebook is present in our Box repository. The cells below download:

1. The association file to be used as input to the pipeline
2. The fits files listed in the association file

In [ ]:
box_path = 'https://stsci.box.com/shared/static/'

In [ ]:
def download_file(url):
    """Download into the current working directory the
    file from Box given the direct URL
    
    Parameters
    ----------
    url : str
        URL to the file to be downloaded
        
    Returns
    -------
    download_filename : str
        Name of the downloaded file
    """
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Wrong URL - {}".format(url))
    download_filename = response.headers['Content-Disposition'].split('"')[1]
    with open(download_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return download_filename

<a id='download_asn'></a>
### Download Association file

In [ ]:
association_file_link = 'jdgpjy52yvvos5kk0108numipfgdnfkj.json'
association_url = os.path.join(box_path, association_file_link)
association_file = download_file(association_url)
print(association_file)

<a id='download_fits'></a>
### Download FITS files

In [ ]:
# Direct links to the FITS files hosted on Box
fits_box_links = ['ps1m5z30oodii33rdzm0w7a4kqypyv6s.fits', 'jf3iukp6bjka2odvqkmph84zgrgw61pg.fits',
                  'dckezlakr8a1aqoagoup9qr69404rult.fits', 'lpzmfxgtadu1krh11pj9emv2tlcbbb08.fits',
                  '844vj9yqjyfeiifwfrcxfrm57k3qftd1.fits', '0nc0g6887hpjp4mu7g75oqzq3mtcn5kk.fits',
                  'pd9k9hm7zbp63b20trnquxzsvkbaijlj.fits', 'p0rzb9njw0nx5p8pjmko9p4i9d1sqrxq.fits',
                  '5r74nm0s1svbpm8frb3v2ljfzwju6yt3.fits', '75jpbzjaqm5owxqhzbz32plu5s28cwj5.fits',
                  'j8gzofqew5m33m090dcvjd5dv7w8bcdy.fits', '6vyp86zmnk78v01w0cklrunfvsogwzh2.fits',
                  'vx1lpbhqg8ohxq8nshgqkwlyp8mnq3ry.fits', '34zrr17waagax636cnmrf59qbbm9ojtz.fits',
                  '83i1pior38yijh8urudkbamqjc77rkay.fits', '32cebe1om7asz5uphygeibjq5pief8xs.fits']

In [ ]:
# Grab a copy of the data used in this notebook from the Box repository
for boxfile in fits_box_links:
    file_url = os.path.join(box_path, boxfile)
    fits_file = download_file(file_url)
    print("Downloading {}".format(fits_file))

***
<a id='pipeline_configs'></a>
### Create Configuration Files

A copy of the pipeline configuration files used in this notebook are included in this repository. For details on configuration files, see the [JWST configuration file documentation page](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html#configuration-files).

Configuration files are optional inputs for each step of the pipeline, as well as for the pipeline itself. These files list step-specific parameters, and can also be used to control which steps are run as part of the pipeline.

You can get the full compliment of configuration files using the `collect_pipeline_cfgs` convenience function from the command line:

>`$ collect_pipeline_cfgs ./`

This creates a copy of all configuration files, for all steps and all JWST Instruments. Note that default parameters in the config files are not necessarily optimized for any particular instrument. For the purposes of this notebook, we will use the default versions of the configuration files.

Each of these configuration files can be customized to control pipeline behavior. For example, the configuration file for the Level 3 imaging pipeline is called **calwebb_image3.cfg** and contains a list (not necessarily in order) of the steps run as part of the Level 3 imaging pipeline.


    name = "Image3Pipeline"
    class = "jwst.pipeline.Image3Pipeline"

        [steps]
          [[tweakreg]]
            config_file = tweakreg.cfg
            skip = True
          [[skymatch]]
            config_file = skymatch.cfg
          [[outlier_detection]]
            config_file = outlier_detection.cfg
          [[resample]]
            config_file = resample.cfg
          [[source_catalog]]
            config_file = source_catalog.cfg
            save_results = true
        
In the example shown above, the ***tweakreg*** step will be skipped (`skip = True`), and the output from the ***source_catalog*** step will be saved (`save_results = True`).

Note that **calwebb_image3.cfg** lists a configuration file for each pipeline step. You can customize a particular pipeline step by editing the parameters in its configuration file. For example, the source catalog configuration file, shown below, contains details on the kernel size and FWHM, as well as the signal to noise threshold to use in the identification of sources in the final combined image. 


    name = "source_catalog"
    class = "jwst.source_catalog.SourceCatalogStep"

    kernel_fwhm = 3.
    kernel_xsize = 5.
    kernel_ysize = 5.
    snr_threshold = 3.
    npixels = 50
    deblend = False

***
<a id='file_info'></a>
### File Information

#### Association Table

Ths first file downloaded above was an association table. This is the input to the calwebb_image3 pipeline, and is a **json** file that contains a list of all the FITS files to be combined into a single mosaic using the calwebb_imaging3 pipeline. An example association table is shown below.

Files that cannot be combined (e.g. NIRCam shortwave and longwave data) must be placed in separate association tables.

#### Example Association Table

The cell below shows an example association table composed of two dithers of the shortwave NIRCam B module. Detectors B1 through B4 comprise the B module. Note that in the filenames below there are two appearances of each of these detectors (e.g. nrcb1).

***
<a id='create_association'></a>
#### Create an Association Table from Scratch

It is possible to create your own association file if you have a list of data files to be calibrated. Currently the best way to do this is using the **asn_from_list** tool on the command line.

>`$ asn_from_list *_cal.fits -o jw1002_level3_asn.json --product-name jw10002_short/`

<a id='fits_info'></a>
#### FITS files

The FITS files downloaded above are from an example NIRCam dataset containing data from the 4 detectors which comprise the shortwave B module. The 16 FITS files correspond to 4 dithers using all 4 detectors.

Each multi extension FITS file contains the following extensions:

    * 0: PRIMARY. Header information only
    * 1: SCI. 2D astronomical source image
    * 2: ERR. 2D error array associated with the source image
    * 3: DQ. 2D array of bad pixel flags 
    * 4: AREA. 2D array containing a normalized pixel area map
    * 5: VAR_POISSON. 2D array of variance values from Poisson noise
    * 6: VAR_RNOISE. 2D array of variance values from readnoise
    * 7: ASDF. Contains WCS information about the observation

***
<a id="methods"></a>
## Methods of calling the pipeline

The 4 methods for calling the pipeline:
1. [Call the pipeline with configuration files](#pipeline_with_cfgs)
2. [Call the pipeline with parameters set programmatically](#pipeline_no_configs)
3. [Call the individual pipeline steps using configuration files](#steps_with_config_files)
4. [Call the individual pipeline steps with parameters set programmatically](#steps_no_configs)

***
<a id="pipeline_with_cfgs"></a>
### Run Pipeline with Configuration Files

Once you have edited the configuration files to customize the Level 3 pipeline, the command below will run the pipeline. This will generate the following products:

 - Final source catalog ***cat.ecsv***
 - Final 2D image ***i2d.fits***
 - Individual exposures with DQ array flagged for outliers ***crf.fits***
 - Individual blotted images from the outlier detection step ***blot.fits***.

***
**WARNING:** when using configuration files, the pipeline must be called using the **call** method. There is also a **run** method (which can be used when calling with [parameters set programmatically](#pipeline_no_configs)). Only the **call** method inspects the configuration files. Any configuration files provided to the **run** method will be ignored!
***

In [ ]:
# Note that the basename of the output combined image is set in the association file
# in the "name" field.
m = Image3Pipeline.call(association_file, config_file='calwebb_image3.cfg')

#### Source Catalog

Note that the filename of the output source catalog and combined image is specified in the `"name"` field in the association file which was input to the pipeline. The basename value in the association table has `_cat.ecsv` appended to create the source catalog filename, and `_i2d.fits` appended to create the output image filename.

In [ ]:
# Examine the source catalog product
catalog = Table.read("jw10002_short_cat.ecsv",format='ascii.ecsv')

In [ ]:
# List columns in the source catalog
print(catalog.colnames)

In [ ]:
# Show the catalog
catalog

#### View the Final Combined 2D Image

In [ ]:
# Combined image
combined_image_file = 'jw10002_short_i2d.fits'
combined_image = fits.getdata(combined_image_file)

In [ ]:
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=0, vmax=50), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="pipeline_no_configs"></a>
### Run Pipeline with Parameters Set Programmatically

You can also run the pipeline without relying on configuration files by setting parameters programmatically, in combination with relying on the defaults in the pipeline. Note that this makes use of the **run** method.

In [ ]:
m = Image3Pipeline()

# You can skip steps and change parameter values.
# Note that the attribute names below match those in the configuration file for the appropriate pipeline step
m.tweakreg.skip = False
m.source_catalog.snr_threshold = 10
m.source_catalog.output_file = 'jw10002_short_no_cfgs_cat.ecsv'
m.save_results = True

# Run the pipeline with these paramters
m.run(association_file)

#### Source Catalog

In this case we specified the output source catalog filename before running the pipeline.

In [ ]:
# Examine source catalog product
catalog = Table.read(m.source_catalog.output_file, format='ascii.ecsv')
catalog

#### Combined Image

In order to access the combined image created by the call to the pipeline above, you must read in the output `i2d` file. The output filename is specified in the association file which was input to the pipeline.

In [ ]:
no_cfg = fits.getdata('jw10002_short_i2d.fits')

In [ ]:
norm = ImageNormalize(no_cfg, interval=ManualInterval(vmin=0, vmax=50), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(no_cfg, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="steps_with_config_files"></a>
### Run Individual Steps with Configuration Files

It is also possible to call the individual pipeline steps, rather than the pipeline itself. Here we show an example of calling individual steps using their configuration files and the **call** method.

***
**WARNING:** when using configuration files, the steps must be called using the **call** method. There is also a **run** method (which can be used when calling with [parameters set programmatically](#steps_no_configs). Only the **call** method inspects the configuration files. Any configuration files provided to the **run** method will be ignored!
***

The level 3 pipeline for imaging data (calwebb_image3) is composed of five steps. 

 1. [tweakreg](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html): Performs image alignment using source catalogs.
 2. [skymatch](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html): Measures and adjusts sky levels in input data so that they are consistent.
 3. [outlier detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/index.html): Searches for transient sources (e.g. cosmic rays) and flags them such that they are ignored during image combination.
 4. [resample](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html): Will resample all individual images, taking into account WCS and distortion information and combines into a single, distortion-free product
 5. [source catalog](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html): Creates a final catalog of sources in the combined 2d image produced in the resample step.

Note how pipeline steps can be chained together: The initial (tweakreg_step) step takes the association file as input. Each subsequent step takes the output of the previous step (along with the appropriate configuration file) as its input.

Each pipeline step outputs an instance of a particular type of [datamodel](https://jwst-pipeline.readthedocs.io/en/stable/jwst/datamodels/models.html). For example, the Resample step outputs an instance of an ImageModel. Use the `data` attribute of a datamodel instance to view the 2D image in the instance. This differs from the `Image3Pipeline` instance, which returns nothing. In that case, the output file must be read in to access the final image.

In [ ]:
tr_step = tweakreg_step.TweakRegStep.call(association_file, config_file='tweakreg.cfg')
sky_step = skymatch_step.SkyMatchStep.call(tr_step, config_file='skymatch.cfg')
out_step = outlier_detection_step.OutlierDetectionStep.call(sky_step, config_file='outlier_detection.cfg')
resamp_step = resample_step.ResampleStep.call(out_step, config_file='resample.cfg',
                                    output_file='jw10002_short_step_by_step_i2d.fits')
sourcecat_step = source_catalog_step.SourceCatalogStep.call(resamp_step, config_file='source_catalog.cfg',
                                               output_file='jw10002_short_step_by_step_cat.ecsv')

Again, we specified the source catalog filename in the call to the SourceCatalogStep above.

In [ ]:
# Examine output catalog
catalog = Table.read('jw10002_short_step_by_step_cat.ecsv',format='ascii.ecsv')
catalog

In this case, you can read in the data from the resampled output file which was specified above: `jw10002_short_step_by_step_i2d.fits`, or you can examine the image using the `data` attribute of the datamodel returned by the `resample` step.

In [ ]:
final_image = resamp_step.data

# OR
# final_image = fits.getdata('jw10002_short_step_by_step_i2d.fits')

In [ ]:
# Look at registered & combined image
f,a = plt.subplots(figsize=(12, 12))
norm = simple_norm(final_image, 'log', min_cut=0, max_cut=50)
img = a.imshow(final_image, norm=norm, origin='lower', cmap='viridis')
f.colorbar(img)
plt.show()

***
<a id="steps_no_configs"></a>
### Run Individual Steps with Parameters Set Programmatically

As with the pipeline example, the individual pipeline steps can also have their parameters set programmatically before being called with the **run** method.

In [ ]:
# Tweakreg
tr = tweakreg_step.TweakRegStep()
tr.save_catlogs = False
tr.snr_threshold = 25.
tr.minobj = 15
tr_results = tr.run(association_file)

In [ ]:
# Skymatch
sm = skymatch_step.SkyMatchStep()
sm.skymethod = 'global+match'
sm.subtract = False
sm.skystat = 'mode'
sm.save_results = False
sm_results = sm.run(tr_results)

In [ ]:
# Outlier Detection
od = outlier_detection_step.OutlierDetectionStep()
od.weight_type = 'exptime'
od.kernel = 'square'
od.snr = '4.0 3.0'
od.save_intermediate_results = False
od.good_bits = 4
od_results = od.run(sm_results)

In [ ]:
# Resample
resamp = resample_step.ResampleStep()
resamp.weight_type = 'exptime'
resamp.good_bits = 4
resamp.save_results = True
resamp_results = resamp.run(od_results)

In [ ]:
# Look at registered & combined image
f,a = plt.subplots(figsize=(12, 12))
norm = simple_norm(resamp_results.data, 'log', min_cut=0, max_cut=50)
img = a.imshow(resamp_results.data, norm=norm, origin='lower', cmap='viridis')
f.colorbar(img)
plt.show()

In [ ]:
# Source Catalog
sc = source_catalog_step.SourceCatalogStep()
sc.snr_threshold = 10
sc.npixels = 50
sc.save_results = True
sourcecat_output_filename = 'jw10002_step_by_step_no_cfg'
sc.output_file = sourcecat_output_filename
sc_results = sc.run(resamp_results)

In [ ]:
# Examine output catalog
output_cat = "{}_{}".format(sourcecat_output_filename, 'cat.ecsv')
catalog = Table.read(output_cat,format='ascii.ecsv')
catalog

***
<a id='addtl_resources'></a>
## Additional Resources

There are several different places to find information on installing and running the pipeline. 

>1. [Data Reduction Pipeline page](https://jwst-docs.stsci.edu/display/JDAT/JWST+Data+Reduction+Pipeline)
>2. [Pipeline Installation page](http://astroconda.readthedocs.io/en/latest/releases.html#pipeline-install)
>3. [Detailed pipeline information](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html)
>4. [JWST Help Desk](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f)

Level 3 pipeline steps are listed below. This notebook gives shortened descriptions of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below.

* **[Tweakreg](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html)** (jwst.tweakreg, tweakreg_step, TweakRegStep)
* **[Sky Match](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html)** (jwst.skymatch, skymatch_step, SkyMatchStep)
* **[Outlier Detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/index.html)** (jwst.outlier_detection, outlier_detection_step, OutlierDetectionStep)
* **[Resample](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html)** (jwst.resample, resample_step, ResampleStep)
* **[Source Catalog](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html)** (jwst.source_catalog, source_catalog_step, SourceCatalogStep)

(for more information on individual steps see the [JWST package index](https://jwst-pipeline.readthedocs.io/en/latest/jwst/package_index.html))

***
<a id='about'></a>
## About this notebook

This notebook was created by Bryan Hilbert (STScI Research and Instrument Scientist), based on other notebooks created by Alicia Canipe (STScI Research and Instrument Analyst). For additional questions about the **calwebb_image3** pipeline or its constituent steps, contact the [JWST Help Desk](https://stsci.service-now.com/jwst).

Updated on 26 November 2018

![img](footer.png)